In [20]:
import requests
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import psycopg2
import smtplib
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from requests.exceptions import ChunkedEncodingError, RequestException
import time

headers = {
    'authority': 'moto.av.by',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru,en;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "YaBrowser";v="24.1", "Yowser";v="2.5"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 YaBrowser/24.1.0.0 Safari/537.36',
}

# Чтение json конфига
with open('config.json') as file:
    config = json.load(file)

mail_login = config['sender login']
mail_password = config['sender password']
pgre_login = config['postgre login']
pgre_password = config['postgre password']
pgre_host = config['postgre host']
pgre_port = config['postgre port']
pgre_db = config['postgre database']
recipients = config['mail recipients']

# Чтение json исключений
with open('exceptions.json', encoding="utf8") as file:
    exceptions_json = json.load(file)

exclude_locations = exceptions_json['exclude_locations']
exclude_brands = exceptions_json['exclude_brands']

#Подключение к postgres
conn = psycopg2.connect(
    host = pgre_host,
    port = pgre_port,
    database = pgre_db,
    user = pgre_login,
    password = pgre_password
)

In [21]:
id = 114239128

In [22]:
# для теста
duplcursor = conn.cursor()
query = """
    SELECT brand, model, year, type, cylinders, capacity, seller, locations, mileage, url
    FROM public.av_full
    WHERE id = %s
""" % id
duplcursor.execute(query)
results = duplcursor.fetchone()

brand = results[0]
model = results[1]
year = results[2]
mtype = results[3]
cylcount = results[4]
capacity = results[5]
seller = results[6]
location = results[7]
mileage = results[8]
url = results[9]

duplcursor.close()

In [23]:
# Функция "ручной" проверки дубликатов
def duplicates_manual_check(brand, model, year, mtype, cylcount, capacity, seller, location, mileage):
    duplcursor = conn.cursor()
    query = """
        SELECT id, url, duplicate_flag
        FROM public.av_full
        WHERE brand = %(brand)s
          AND model = %(model)s
          AND year = %(year)s
          AND type = %(type)s
          AND cylinders = %(cylinders)s
          AND capacity = %(capacity)s
          AND LOWER(seller) = %(seller)s
          AND locations = %(location)s
          AND mileage BETWEEN %(min_mileage)s AND %(max_mileage)s
          AND duplicate_flag is false
        AND id != %(id)s
        ORDER BY date ASC;
    """
    params = {
        'id': id,
        'min_mileage': mileage * 0.7,
        'max_mileage': mileage * 1.3,
        'brand': brand, 
        'model': model,
        'year': year,
        'type': mtype,
        'cylinders': cylcount,
        'capacity': capacity,
        'seller':seller.lower(),
        'location': location
    }
    duplcursor.execute(query, params)
    dmc_results = duplcursor.fetchall()

    duplcursor.close()

    return dmc_results

In [24]:
# Запуск функции проверки дубликатов
if int(capacity) >= 299 and cylcount > 1 and brand not in exclude_brands:
    manual_duplicate_results = duplicates_manual_check(brand, model, year, mtype, cylcount, capacity, seller, location, mileage)
else:
    manual_duplicate_results = []


In [25]:
# Дополнение хтмл отчета строчкой про дубликаты если они есть
if len(manual_duplicate_results) > 0:
    duplicate_html_block = f"""<p style="text-align: center;"><strong>Найдены вероятные дубликаты:</strong></p>"""
    duplicate_html_block += f"""<a href="{url}">{id}</a>,""" # добавить сначала саму объяву
    # Перебор результатов и проверка наличия дубликатов
    for x in manual_duplicate_results:
        m_d_id = x[0]
        m_d_url = x[1]
        m_d_df = x[2]
        duplicate_html_block += f"""<a href="{m_d_url}">{m_d_id}</a>,"""
else:
    duplicate_html_block = f"""<p style="text-align: center;"><strong>Не найдено дубликатов</strong></p>"""

In [26]:
# обнулить html
html_mail_contents = f"""
<html>
        <body>
<table style="width: 950px; height: 160px;" border="1">
<tbody>
<tr style="height: 10px;">
<td style="width: 484.953px; height: 10px;">
<p style="text-align: left;"><strong>№ processed_ads</strong></p>
<p style="text-align: left;">{id}</p>
</td>
<td style="width: 108.281px; height: 10px;">
<p style="text-align: center;">{year} г.в.&nbsp;</p>
</td>
<td style="width: 173.438px; height: 10px; text-align: center;" colspan="3">
<p style="text-align: center;"><a href="{url}"> <strong>{brand} {model} modification</strong></a></p>
</td>
</tr>
<tr style="height: 35px;">
<td style="width: 484.953px; height: 78px;" rowspan="3"><img src="img_src" alt="" /></td>
<td style="width: 108.281px; height: 35px;">
<p style="text-align: left;"><strong>Актуальное влк</strong></p>
<p style="text-align: left;">mvlk_actual</p>
</td>
<td style="width: 173.438px; height: 35px;" colspan="3">
<p>{mtype}</p>
<p>{cylcount} цилиндров</p>
<p>{capacity} см3</p>
<p>{mileage} км.</p>
<p>condition</p>
<p>Флаг exclude - exclude_flag_actual</p>
</td>
</tr>
<tr style="height: 33px;">
<td style="width: 108.281px; height: 33px;">
<p style="text-align: left;"><strong>Лучшее влк</strong></p>
<p style="text-align: left;">best_match</p>
</td>
<td style="width: 173.438px; height: 33px;" colspan="3">
<p style="text-align: center;"><strong>Продавец</strong></p>
<p style="text-align: center;">{seller}</p>
<p style="text-align: center;"><strong>Локация</strong></p>
<p style="text-align: center;">{location}</p>
{duplicate_html_block}
</td>
</tr>
<tr style="height: 10px;">
<td style="height: 10px; text-align: center; width: 287.719px;" colspan="4">
<blockquote><strong>Дата апдейта&nbsp;</strong>refresh_for_print</blockquote>
<strong>Дата подачи </strong>publish_for_print</td>
</tr>
<tr style="height: 62px;">
<td style="text-align: center; height: 62px; width: 484.953px;">
<h4><strong>price USD</strong></h4>
</td>
<td style="text-align: center; width: 108.281px;" colspan="4"><strong>Ценовая статистика согласно актуальному vlk:</strong><br />Средняя по актуальным = price_a, разница = <span style="color: price_color_a;">price_dif_fr_act</span><br />Средняя за все время = price_f, разница = <span style="color: price_color_f;">price_dif_fr_full</span><strong><br /></strong></td>
</tr>
</tbody>
</table>
<p>&nbsp;</p>
        </body>
        </html>
        """

In [27]:
print(html_mail_contents)


<html>
        <body>
<table style="width: 950px; height: 160px;" border="1">
<tbody>
<tr style="height: 10px;">
<td style="width: 484.953px; height: 10px;">
<p style="text-align: left;"><strong>№ processed_ads</strong></p>
<p style="text-align: left;">114239128</p>
</td>
<td style="width: 108.281px; height: 10px;">
<p style="text-align: center;">2018 г.в.&nbsp;</p>
</td>
<td style="width: 173.438px; height: 10px; text-align: center;" colspan="3">
<p style="text-align: center;"><a href="https://moto.av.by/bike/kawasaki/ninja/114239128"> <strong>Kawasaki Ninja modification</strong></a></p>
</td>
</tr>
<tr style="height: 35px;">
<td style="width: 484.953px; height: 78px;" rowspan="3"><img src="img_src" alt="" /></td>
<td style="width: 108.281px; height: 35px;">
<p style="text-align: left;"><strong>Актуальное влк</strong></p>
<p style="text-align: left;">mvlk_actual</p>
</td>
<td style="width: 173.438px; height: 35px;" colspan="3">
<p>спорт</p>
<p>2 цилиндров</p>
<p>400 см3</p>
<p>12731 

In [12]:
print(duplicate_html_block)

NameError: name 'duplicate_html_block' is not defined

In [152]:
conn.close()